In [ ]:
from google.colab import drive
drive.mount('/content/Gdrive')

In [ ]:
cd /content/Gdrive/My Drive/LIDC-IDRI/

In [ ]:
#FilePaths
dir= '/content/Gdrive/My Drive/LIDC-IDRI/'
datafolder=dir+'ProcessedData'
metafile=dir+"LIDC-IDRI_MetaData.csv"
list32file=dir+"list3.2.csv"
DOIpath=dir+'Dataset/'
unetweightspath=dir+"modelweights/unet-weights-improvement.hdf5"
truenoduleweightspath=dir+"modelweights/truenodule-cnn-weights-improvement.hdf5"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dicom
import os
import scipy.ndimage
import time
from keras.callbacks import ModelCheckpoint
import h5py
from sklearn.cluster import KMeans
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import cell_magic_wand as cmw
from glob import glob
import random
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, SpatialDropout2D
from keras.layers import Input, merge, UpSampling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.models import Sequential
from keras import backend as K
from keras.optimizers import Adam
K.set_image_dim_ordering('th') 
nodulelocations=pd.read_csv(list32file)
meta=pd.read_csv(metafile)

meta=meta.drop(meta[meta['Modality']!='CT'].index)
meta=meta.reset_index()

In [ ]:
#Get folder names of CT data for each patient
patients=[DOIpath+meta['Patient Id'][i] for i in range(len(meta))]
datfolder=[]
for i in range(0,len(meta)-1):
    for path in os.listdir(patients[i]):
        if os.path.exists(patients[i]+'/'+path+'/'+meta['Series UID'][i]):
            datfolder.append(patients[i]+'/'+path+'/'+meta['Series UID'][i])
patients=datfolder

In [ ]:
#Load nodules locations
smooth = 1.0
width = 32

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def unet_model():
    inputs = Input((1, 512, 512))
    conv1 = Conv2D(width, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Conv2D(width, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(width*2, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = BatchNormalization(axis = 1)(conv2)
    conv2 = Conv2D(width*2, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(width*4, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = BatchNormalization(axis = 1)(conv3)
    conv3 = Conv2D(width*4, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(width*8, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = BatchNormalization(axis = 1)(conv4)
    conv4 = Conv2D(width*8, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(width*16, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = BatchNormalization(axis = 1)(conv5)
    conv5 = Conv2D(width*16, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = SpatialDropout2D(0.35)(up6)
    conv6 = Conv2D(width*8, 3, 3, activation='relu', border_mode='same')(conv6)
    conv6 = Conv2D(width*8, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = SpatialDropout2D(0.35)(up7)
    conv7 = Conv2D(width*4, 3, 3, activation='relu', border_mode='same')(conv7)
    conv7 = Conv2D(width*4, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = SpatialDropout2D(0.35)(up8)
    conv8 = Conv2D(width*2, 3, 3, activation='relu', border_mode='same')(conv8)
    conv8 = Conv2D(width*2, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = SpatialDropout2D(0.35)(up9)
    conv9 = Conv2D(width, 3, 3, activation='relu', border_mode='same')(conv9)
    conv9 = Conv2D(width, 3, 3, activation='relu', border_mode='same')(conv9)
    conv10 = Conv2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
    return model

model=unet_model()
model.load_weights(unetweightspath)

In [ ]:
# Load the scans in given folder path
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s, force=True) for s in os.listdir(path) if s.endswith('.dcm')]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]), reverse=True)
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

#convert to ndarray
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

def processimage(img):
    #Standardize the pixel values
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    #plt.hist(img.flatten(),bins=200)
    #plt.show()
    #print(thresh_img[366][280:450])
    middle = img[100:400,100:400] 
    mean = np.mean(middle)  
    max = np.max(img)
    min = np.min(img)
    #move the underflow bins
    img[img==max]=mean
    img[img==min]=mean
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image
    eroded = morphology.erosion(thresh_img,np.ones([4,4]))
    dilation = morphology.dilation(eroded,np.ones([10,10]))
    labels = measure.label(dilation)
    label_vals = np.unique(labels)
    #plt.imshow(labels)
    #plt.show()
    labels = measure.label(dilation)
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0]<475 and B[3]-B[1]<475 and B[0]>40 and B[2]<472:
            good_labels.append(prop.label)
    mask = np.ndarray([512,512],dtype=np.int8)
    mask[:] = 0
    #
    #  The mask here is the mask for the lungs--not the nodes
    #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask 
    #
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation
    return mask*img

def processimagefromfile(ppix):
    processpix=np.ndarray([ppix.shape[0],512,512])
    for i in range(ppix.shape[0]):
        processpix[i]=processimage(ppix[i])
    return processpix

#predict mask from images
def predictmask(images):
    images=images.reshape(images.shape[0],1,512,512)
    num_test=images.shape[0]
    imgs_mask_test = np.ndarray([num_test,1,512,512],dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = model.predict([images[i:i+1]], verbose=0)[0]
    return imgs_mask_test

#find number of slices where a nodule is detected
def getnoduleindex(imgs_mask_test):
    masksum=[np.sum(maskslice[0]) for maskslice in imgs_mask_test]
    return [i for i in range(len(masksum)) if masksum[i]>5]

def nodule_coordinates(nodulelocations,meta):
    slices=nodulelocations["slice no."][nodulelocations.index[nodulelocations["case"]==int(meta["Patient Id"][-4:])]]
    xlocs=nodulelocations["x loc."][nodulelocations.index[nodulelocations["case"]==int(meta["Patient Id"][-4:])]]
    ylocs=nodulelocations["y loc."][nodulelocations.index[nodulelocations["case"]==int(meta["Patient Id"][-4:])]]
    nodulecoord=[]
    for i in range(len(slices)):
        nodulecoord.append([slices.values[i]-1,xlocs.values[i]-1,ylocs.values[i]-1])
    return nodulecoord

#generate nodule or non-nodule labels for mask predictions
def truenodules(noduleindex,masks,nodulecoords):
    label=[]
    for ind in noduleindex:
        for cord in nodulecoords:
            if abs(ind-cord[0])<2:
                com=scipy.ndimage.center_of_mass(masks[ind])
                if abs(com[1]-cord[2])<2 and abs(com[2]-cord[1])<2:
                    label.append(True)
            else:
                label.append(False)
    return label
    
def slicecount(start,end):
    slicecounts=[]
    for i in range(start,end):
        if len(nodule_coordinates(nodulelocations,meta.iloc[i]))>0:
            patient_scan=load_scan(patients[i])
            slicecounts.append(len(patient_scan))
    return slicecounts

In [ ]:
start_time=time.time()
elapsed_time=0
totaltime=50000
noduleimages=np.ndarray([10000,512,512],dtype=np.float32)
nodulelabels=[]
nodulesensitivity=[]
index=0
coordcount=0
coordtruecount=0
nodulecount=0
noduletruecount=0
slicecounts=[]
thresh=-500 #lower HU threshold for nodule segmentation
for i in range(197,400):
    print("Processing patient#",i,"ETA:",(totaltime-elapsed_time)/3600,"hrs")
    if len(nodule_coordinates(nodulelocations,meta.iloc[i]))>0:
        patient_scan=load_scan(patients[i])
        slicecounts.append(len(patient_scan))
        patient_pix=get_pixels_hu(patient_scan)
        processed_pix = processimagefromfile(patient_pix)
        coord = nodule_coordinates(nodulelocations,meta.iloc[i])
        print(coord)
        radius = nodulelocations["eq. diam."][nodulelocations.index[nodulelocations["case"]==int(meta["Patient Id"][i][-4:])]]
        print(radius)
        mask = predictmask(processed_pix)
        noduleindex = getnoduleindex(mask)        
        labels = np.zeros(len(noduleindex)).astype(np.bool)
        cordlabels=np.zeros(len(coord)).astype(np.bool)
        for j,cord in enumerate(coord): #loop through labeled nodules
            if radius.iloc[j]>5:
                nodulemask = cmw.cell_magic_wand(-patient_pix[int(cord[0])],[int(cord[2]),int(cord[1])],2,int(radius.iloc[j])+2)
                nodulepix=nodulemask*patient_pix[cord[0]]
                nodulepix[nodulepix<thresh]=0
                nodulepix[nodulepix!=0]=1
                nodulemask=nodulepix.astype(np.bool)
                for k,ind in enumerate(noduleindex): #loop through detected nodules
                    if abs(ind-cord[0])<2:
                        if np.sum(nodulemask*mask[ind][0])>1:
                            print("Nodule Detected at slice#",ind,"with actual coord",cord)
                            labels[k] = True
                            cordlabels[j] = True

        for j in range(len(coord)):
            nodulesensitivity.append(cordlabels[j])
            
        for k in range(len(noduleindex)):
            nodulelabels.append(labels[k])
            noduleimages[index]=processed_pix[noduleindex[k]]
            index+=1
        coordtruecount+=len(cordlabels[cordlabels==True])
        noduletruecount+=len(labels[labels==True])
        coordcount+=len(coord)
        nodulecount+=len(noduleindex)
        print("Sensitivity:",coordtruecount/coordcount)
        print("Specificity:",noduletruecount/nodulecount)
    elapsed_time=time.time()-start_time
    totaltime=elapsed_time/(i+1)*200
print(elapsed_time)
noduleimages=noduleimages[:len(nodulelabels)]

In [ ]:
noduleimages=noduleimages[:len(nodulelabels)]
noduleimages=noduleimages.reshape([noduleimages.shape[0],1,512,512])
nodulelabels=np.array(nodulelabels)
np.save("noduleimages197-399.npy",noduleimages)
np.save("nodulelabels197-399.npy",nodulelabels)

In [ ]:
nodulelabels=np.load("nodulelabels0-192.npy")
nodulelabels=np.concatenate((nodulelabels,np.load("nodulelabels197-399.npy")))

In [ ]:
TP=len([nl for nl in nodulelabels if nl==True])
FP=len([nl for nl in nodulelabels if nl==False])
print("Number of True Positive nodules:",TP)
print("Number of False Positive nodules:",FP)
print("# of FPs per TP",FP/TP)

In [ ]:
noduleimagestrue=np.load("noduleimagesv3.npy")
falseind=[i for i in range(len(nodulelabels)) if nodulelabels[i]==False]
random.shuffle(falseind)
falseind=falseind[:noduleimagestrue.shape[0]]
noduleimagesfalse=np.array([noduleimages[i] for i in falseind])
del noduleimages
noduleimagesbalanced=np.concatenate((noduleimagestrue,noduleimagesfalse.reshape(noduleimagesfalse.shape[0],1,512,512)),axis=0)
nodulelabelsbalanced=[True]*noduleimagestrue.shape[0]+[False]*noduleimagesfalse.shape[0]
nodulelabelsbalanced=to_categorical(nodulelabelsbalanced,2)
del noduleimagestrue,noduleimagesfalse
Xtrain, Xtest, Ytrain, Ytest = train_test_split(noduleimagesbalanced,nodulelabelsbalanced,test_size=.30)
del noduleimagesbalanced, nodulelabelsbalanced

In [ ]:
#classify as nodule or non-nodule
input_shape=(1,512,512)
num_classes=2
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

checkpoint = ModelCheckpoint(truenoduleweightspath, monitor='loss', verbose=1, save_best_only=True)

history=model.fit(Xtrain, Ytrain, batch_size=4, epochs=20, verbose=1, shuffle=True, callbacks=[checkpoint], validation_data=(Xtest,Ytest))

In [ ]:
plt.plot(history.history['loss'], color='b')
plt.plot(history.history['val_loss'], color='g')
plt.xlabel("Epoch")
plt.ylabel("Log Loss")
plt.legend(["Train", "Validation"])
plt.show()
plt.plot(history.history['acc'], color='b')
plt.plot(history.history['val_acc'], color='g')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["Train", "Validation"])
plt.show()

In [ ]:
XtestTrue=np.array([Xtest[i] for i in range(Ytest.shape[0]) if Ytest[i,1]==1])
YtestTrue=np.array([Ytest[i] for i in range(Ytest.shape[0]) if Ytest[i,1]==1])
XtestFalse=np.array([Xtest[i] for i in range(Ytest.shape[0]) if Ytest[i,1]==0])
YtestFalse=np.array([Ytest[i] for i in range(Ytest.shape[0]) if Ytest[i,1]==0])

scoretrue=model.evaluate(XtestTrue,YtestTrue)